In [1]:
import pyspark
from pyspark import SparkContext, SparkConf
from datetime import datetime

In [2]:
#это нужно для работы с файлами типа xml
from pyspark.sql import SparkSession
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.14.0 pyspark-shell'

In [3]:
sc = SparkSession.builder.appName("Project2").master("local[*]").getOrCreate()

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [4]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType

#Считываем названия языков программирования
prog_languages = sc.read.csv("/mnt/data/programming-languages.csv")

#Создаём из названий список, так удобнее
prog_lang_name=[]
for x in prog_languages.collect():
    prog_lang_name.append(str(x[0]))
prog_lang_name[0:10]

['name',
 'A# .NET',
 'A# (Axiom)',
 'A-0 System',
 'A+',
 'A++',
 'ABAP',
 'ABC',
 'ABC ALGOL',
 'ABSET']

In [5]:
#Считываем данные, которые нужно будет анализировать, выбираем row, который будет интепретироваться как строка в Spark
PSample = sc.read.format('xml').option('rowTag', 'row').load("/mnt/data/posts_sample.xml")

In [6]:
PSample.first()

Row(_AcceptedAnswerId=7, _AnswerCount=13, _Body="<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n  <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code></p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n", _ClosedDate=None, _CommentCount=2, _CommunityOwnedDate=datetime.datetime(2012, 10, 31, 16, 42, 47, 213000), _CreationDate=datetime.datetime(2008, 7, 31, 21, 42, 52, 667000), _FavoriteCount=48, _Id=4, _LastActivityDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditorDisplayName='Rich B', _LastEditorUserId=3641067, _OwnerDisplayName=None, _OwnerUse

In [7]:
#переходим к rdd потому,что так мне удобнее и привычнее работать

PS_RDD=PSample.rdd
PS_RDD.take(1)

[Row(_AcceptedAnswerId=7, _AnswerCount=13, _Body="<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n  <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code></p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n", _ClosedDate=None, _CommentCount=2, _CommunityOwnedDate=datetime.datetime(2012, 10, 31, 16, 42, 47, 213000), _CreationDate=datetime.datetime(2008, 7, 31, 21, 42, 52, 667000), _FavoriteCount=48, _Id=4, _LastActivityDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditorDisplayName='Rich B', _LastEditorUserId=3641067, _OwnerDisplayName=None, _OwnerUs

In [8]:
#Что бы найти самые популярные языки по годам, будем считать сколько раз они упоминались в тегах
#Для начала исключим года выходящие за установленные рамки ([2010-2020])
#Для этого пишем метод checking_the_date
def checking_the_date(row):
    return 2009 < row._CreationDate.year and row._CreationDate.year < 2021

In [9]:
#Будем последовательно совершать все необходимые шаги
#Исключаем года выходящие за рамки
d2=PS_RDD.filter(lambda row: checking_the_date(row))

In [10]:
d2.take(3)

[Row(_AcceptedAnswerId=None, _AnswerCount=None, _Body='<p>No. (And more stuff to round this up to 15 characters...)</p>\n', _ClosedDate=None, _CommentCount=6, _CommunityOwnedDate=None, _CreationDate=datetime.datetime(2010, 9, 20, 16, 18, 14, 580000), _FavoriteCount=None, _Id=3753373, _LastActivityDate=datetime.datetime(2010, 9, 20, 16, 18, 14, 580000), _LastEditDate=None, _LastEditorDisplayName=None, _LastEditorUserId=None, _OwnerDisplayName=None, _OwnerUserId=10293, _ParentId=3753364, _PostTypeId=2, _Score=13, _Tags=None, _Title=None, _ViewCount=None),
 Row(_AcceptedAnswerId=None, _AnswerCount=None, _Body='<p>I am not aware of a way to use the URL parameters the way you have indicated. If anyone knows better, do correct me.</p>\n\n<p>I faced a similar situation some time ago and made do with a thin wrapper over the <code>list_detail</code> view. </p>\n\n<pre><code># views.py\nfrom django.views.generic.list_detail import object_list\n\ndef object_list_wrapper(*args, **kwargs):\n    cat

In [11]:
#метод для проверки тегов на наличие в них названия языка
def search_program_language(row):
    tags_lan=None
    for n in prog_lang_name:
        if ('<' + n.lower() + '>') in str(row._Tags).lower():
            tags_lan = n
            break
    if tags_lan is None:
        return (row[6], 'nothing')
    return (row[6], tags_lan)
    

In [12]:
#Отмечаем где есть и где нету упоминаний о названии языка в тегах 
d3=d2.map(search_program_language)

In [13]:
d3.take(5)

[(datetime.datetime(2010, 9, 20, 16, 18, 14, 580000), 'nothing'),
 (datetime.datetime(2010, 9, 20, 18, 36, 28, 23000), 'nothing'),
 (datetime.datetime(2010, 9, 20, 19, 4, 13, 587000), 'nothing'),
 (datetime.datetime(2010, 9, 21, 2, 7, 51, 100000), 'nothing'),
 (datetime.datetime(2010, 9, 21, 7, 33, 58, 707000), 'nothing')]

In [14]:
#Отсеиваем те, где в тэгах не упоминается название языка
d4=d3.filter(lambda row: row[1]!='nothing')

In [15]:
d4.take(5)

[(datetime.datetime(2010, 9, 23, 12, 13, 59, 443000), 'Java'),
 (datetime.datetime(2010, 9, 26, 17, 7, 4, 840000), 'PHP'),
 (datetime.datetime(2010, 9, 30, 18, 27, 56, 320000), 'Ruby'),
 (datetime.datetime(2010, 10, 1, 11, 52, 42, 210000), 'C'),
 (datetime.datetime(2010, 10, 4, 21, 5, 50, 150000), 'PHP')]

In [16]:
#группируем по году и названию. А затем считаем количество упоминаний
d5=d4.keyBy(lambda row: (row[0].year,row[1])).aggregateByKey(
        0,
        lambda acc, value: acc + 1,
        lambda acc1, acc2: acc1 + acc2,
    )

In [17]:
d5.take(5)

[((2010, 'Python'), 25),
 ((2010, 'JavaScript'), 44),
 ((2010, 'R'), 3),
 ((2011, 'Objective-C'), 33),
 ((2011, 'JavaScript'), 82)]

In [18]:
#Сортируем по убыванию количества упоминаний
d6=d5.sortBy(lambda row: row[1], ascending=False)

In [19]:
d6.take(10)

[((2016, 'JavaScript'), 272),
 ((2015, 'JavaScript'), 270),
 ((2017, 'JavaScript'), 244),
 ((2014, 'JavaScript'), 235),
 ((2014, 'Java'), 228),
 ((2018, 'Python'), 214),
 ((2015, 'Java'), 208),
 ((2017, 'Java'), 204),
 ((2013, 'JavaScript'), 196),
 ((2018, 'JavaScript'), 196)]

In [20]:
#Сортируем по годам
d7=d6.sortBy(lambda row: row[0][0], ascending=False)
d7.take(10)

[((2019, 'Python'), 162),
 ((2019, 'JavaScript'), 131),
 ((2019, 'Java'), 95),
 ((2019, 'PHP'), 59),
 ((2019, 'R'), 36),
 ((2019, 'C'), 14),
 ((2019, 'Go'), 9),
 ((2019, 'Dart'), 9),
 ((2019, 'MATLAB'), 9),
 ((2019, 'Ruby'), 8)]

In [21]:
#Теперь я хочу избавиться от сдвоенного ключа, и добавить столбец с удобной нумерацией,
#которая для каждого года начинается с начала
#Поэтому подготовлю данные и потом создам новый DataFrame
prname=[]
prname2=[]
prname3=[]
prname4=[]

tsr=100000
k=1

for x in d7.collect():
    prname.append(x[0])
    prname3.append(int(x[1]))

for x in prname:
    prname2.append([x[0],x[1]])
    
for x1 in range(0,len(prname)-1):
    t=prname3[x1]
    if(tsr<t):
        k=1
    prname4.append([prname2[x1][0],prname2[x1][1],t,int(k)])
    tsr=t
    k=k+1
prname4[0:10]

[[2019, 'Python', 162, 1],
 [2019, 'JavaScript', 131, 2],
 [2019, 'Java', 95, 3],
 [2019, 'PHP', 59, 4],
 [2019, 'R', 36, 5],
 [2019, 'C', 14, 6],
 [2019, 'Go', 9, 7],
 [2019, 'Dart', 9, 8],
 [2019, 'MATLAB', 9, 9],
 [2019, 'Ruby', 8, 10]]

In [22]:
#создаем новый DataFrame
from pyspark.sql import Row
R = Row('Year','Name','Count','Number')
new_DF=sc.createDataFrame([R(i,x,y,z) for i,x,y,z in (prname4)])


In [26]:
#Выводим чтобы полюбоваться на то, что уже есть
new_DF.show(40)

+----+-----------+-----+------+
|Year|       Name|Count|Number|
+----+-----------+-----+------+
|2019|     Python|  162|     1|
|2019| JavaScript|  131|     2|
|2019|       Java|   95|     3|
|2019|        PHP|   59|     4|
|2019|          R|   36|     5|
|2019|          C|   14|     6|
|2019|         Go|    9|     7|
|2019|       Dart|    9|     8|
|2019|     MATLAB|    9|     9|
|2019|       Ruby|    8|    10|
|2019| PowerShell|    8|    11|
|2019|       Bash|    8|    12|
|2019|     Kotlin|    8|    13|
|2019|      Scala|    6|    14|
|2019| TypeScript|    6|    15|
|2019|     Delphi|    4|    16|
|2019|        AWK|    4|    17|
|2019|       Curl|    3|    18|
|2019|         F#|    3|    19|
|2019|       Perl|    3|    20|
|2019|      LaTeX|    2|    21|
|2019|       Rust|    2|    22|
|2019|     Prolog|    1|    23|
|2019|    NetLogo|    1|    24|
|2019|       NSIS|    1|    25|
|2019|       NASM|    1|    26|
|2019|      XPath|    1|    27|
|2019|    Haskell|    1|    28|
|2019|Ob

In [27]:
#Выбираем топ 10 языков для каждого года
new_DF.createOrReplaceTempView("Table")
top_10 = sc.sql("SELECT Year,Name,Count FROM Table WHERE Number<=10 ")
top_10.show(top_10.count())

+----+-----------+-----+
|Year|       Name|Count|
+----+-----------+-----+
|2019|     Python|  162|
|2019| JavaScript|  131|
|2019|       Java|   95|
|2019|        PHP|   59|
|2019|          R|   36|
|2019|          C|   14|
|2019|         Go|    9|
|2019|       Dart|    9|
|2019|     MATLAB|    9|
|2019|       Ruby|    8|
|2018|     Python|  214|
|2018| JavaScript|  196|
|2018|       Java|  145|
|2018|        PHP|   99|
|2018|          R|   63|
|2018|          C|   24|
|2018|      Scala|   22|
|2018| TypeScript|   21|
|2018| PowerShell|   13|
|2018|       Bash|   12|
|2017| JavaScript|  244|
|2017|       Java|  204|
|2017|     Python|  185|
|2017|        PHP|  122|
|2017|          R|   53|
|2017|          C|   24|
|2017|Objective-C|   19|
|2017|       Ruby|   16|
|2017| PowerShell|   14|
|2017| TypeScript|   14|
|2016| JavaScript|  272|
|2016|       Java|  179|
|2016|     Python|  141|
|2016|        PHP|  126|
|2016|          R|   50|
|2016|          C|   32|
|2016|       Ruby|   21|


In [28]:
#И сохраняем отчёт в формате Apache Parquet.
top_10.write.parquet("top_10_lang.parquet")